##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2021-08-27 18:04:27.189546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 18:04:27.219608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 18:04:27.219862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Show the available GPUs on the system

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2021-08-27 18:04:53.766665: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-27 18:04:53.767583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 18:04:53.767929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 18:04:53.768173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.35981542,  0.05897278, -0.42001423,  0.3027846 , -0.33174652,
        -0.43421462,  0.70751727,  0.456143  ,  0.59307146,  0.98296314]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.05277017, 0.0802169 , 0.04968719, 0.10236511, 0.05427235,
        0.0489866 , 0.15343525, 0.11933138, 0.1368428 , 0.2020923 ]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

3.0162086

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
   1/1875 [..............................] - ETA: 5:10 - loss: 2.3568 - accuracy: 0.0625

2021-08-27 18:05:10.708406: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1875/1875 [==============================] - 2s 773us/step - loss: 0.2969 - accuracy: 0.9146
Epoch 2/5
1875/1875 [==============================] - 1s 674us/step - loss: 0.1439 - accuracy: 0.9572
Epoch 3/5
1875/1875 [==============================] - 1s 675us/step - loss: 0.1076 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 1s 672us/step - loss: 0.0874 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 1s 670us/step - loss: 0.0727 - accuracy: 0.9772


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [14]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0782 - accuracy: 0.9763


[0.07824654877185822, 0.9763000011444092]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.01995221e-09, 1.61593334e-08, 3.55858947e-06, 1.09741559e-05,
        1.15085441e-10, 8.99335362e-09, 1.22441696e-14, 9.99983907e-01,
        3.51827611e-09, 1.54034558e-06],
       [4.90220031e-10, 2.17006964e-04, 9.99774992e-01, 2.67446489e-06,
        1.18989318e-13, 5.22619985e-06, 8.62784155e-09, 3.50514377e-13,
        1.72831037e-07, 7.48849089e-14],
       [6.65199593e-08, 9.99184668e-01, 1.63963177e-05, 9.10052449e-07,
        8.02796130e-05, 8.16444845e-06, 2.15875934e-06, 5.27139462e-04,
        1.79097202e-04, 1.06221228e-06],
       [9.99803960e-01, 5.55497124e-08, 4.76783935e-05, 5.98609290e-07,
        1.44631822e-05, 1.66940790e-05, 8.10340134e-05, 3.00555039e-05,
        2.33689619e-08, 5.59512318e-06],
       [1.61969869e-08, 6.01721917e-09, 1.36555923e-06, 1.97476222e-08,
        9.98564541e-01, 8.48448956e-09, 1.74014502e-07, 3.74281699e-05,
        8.72926194e-08, 1.39634102e-03]], dtype=float32)>

To run in tensorman: 

In [3]:
#tensorman run --gpu python -- ./tensorflow_demo.py

SyntaxError: invalid syntax (2398047115.py, line 1)